This notebook prepares CDB population data for use in propensity score matching.

As a start, an attempt is made to attach the CDB socio-economic data (in spreadsheet format) to the village point shapefile.

## setup

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

<IPython.core.display.Javascript object>

In [3]:
code_path = Path().absolute()
datafd_path = code_path.parent.parent / 'data'
outfd_path = code_path.parent.parent / 'output'

## read in data

### CDB socio-economic data, Reem updated 20230207

In [4]:
CDB_path = datafd_path / 'other' / 'CDB Nat Data 2016 En_Received Dec2017_230207_Reem.xlsx'
# CDB_df_dic = pd.read_excel(CDB_path, sheet_name=None)  # all sheets, key=sheet name
CDB_df_dic = pd.read_excel(CDB_path, sheet_name=['V_2016_E(1)', 'C_2016_E(1)'])
sht_name_lst = list(CDB_df_dic.keys())
# sheet names: ['Q_2016_E', 'V_2016_E(1)', 'V_2016_E(2)', 'V_2016_E(3)', 'C_2016_E(1)', 'C_2016_E(2)', 'D_2016_E(1)']

##### village-level CDB data

In [5]:
vill_sht_name_lst = [sht for sht in sht_name_lst if sht.startswith('V') ]
CDB_v_df_lst = [CDB_df_dic[sht] for sht in vill_sht_name_lst]

##### commune-level CDB data

In [6]:
# comm_sht_name_lst = [sht for sht in sht_name_lst if sht.startswith('C') ]
# CDB_c_df_lst = [CDB_df_dic[sht] for sht in comm_sht_name_lst]

### shapefiles associated with CDB socio-economic data (above), Lok sent 20230407

In [7]:
CDB_shp_fd_path = datafd_path / 'boundaries' / 'Cambodia_Admin-2015'

#### village points

In [8]:
v_pt_gdf = gpd.read_file(CDB_shp_fd_path / 'Villages.shp')  # EPSG:32648

#### commune boundaries

In [9]:
# c_bnd_gdf = gpd.read_file(CDB_shp_fd_path / 'Commune Boundary.shp')  # EPSG:32648

## preprocess data

##### fix headers of village-level CDB data

In [10]:
CDB_v_df_lst = [df.rename(columns=df.iloc[3]).tail(-4) for df in CDB_v_df_lst]

##### fix headers of commune-level CDB data

In [11]:
# CDB_c_df_lst = [df.rename(columns=df.iloc[3]).tail(-4) for df in CDB_c_df_lst]

## explore data

### in order to join village points with village-level CDB data

#### CDB

In [12]:
CDB_v_df0 = CDB_v_df_lst[0]

In [13]:
CDB_v_df0.head(2)

VillGis          Province   District    Commune       Village 1-FAMILY  \
4  1100201  Banteay Meanchey  Paoy Paet  Paoy Paet  Kbal Spean 1     1914   
5  1100202  Banteay Meanchey  Paoy Paet  Paoy Paet     Baliley 1      998   

  3-MAL_TOT 2-FEM_TOT 73-F_HHH 201-MAL_0_2  ... 1176-Fe_Sel_Furniture  \
4      4000      4203       59         189  ...                     1   
5      1916      2254      117         138  ...                     0   

  1177-Const_Material 1178-T-Worker_Const 1179-Fe_Const_Material  \
4                   2                   7                      4   
5                  20                  51                     22   

  1180-Station_Sports 1181-T-Worker_S_S 1182-Fe_Station_Sports  \
4                   1                 2                      1   
5                   9                17                     13   

  1183-Hair_Services 1184-T-Worker_Hair 1185-Fe_Hair_Services  
4                 20                 47                    24  
5                 18                 41                    29  

[2 rows x 255 columns]

In [14]:
CDB_v_df0.isna().any()  # no NA in first 5 columns

VillGis                   False
Province                  False
District                  False
Commune                   False
Village                   False
                          ...  
1181-T-Worker_S_S          True
1182-Fe_Station_Sports     True
1183-Hair_Services         True
1184-T-Worker_Hair         True
1185-Fe_Hair_Services      True
Length: 255, dtype: bool

In [15]:
CDB_v_df0.VillGis.duplicated().any() # no duplicate VillGis

False

#### village points

In [45]:
v_pt_gdf.shape

(13970, 7)

In [16]:
v_pt_gdf.head(2)

NUMBER  PHUMCODE  VILLAGE   PHUM   XPHUM    YPHUM  \
0       1   1020101  Ou Thom  GUrFM  287800  1494700   
1       2   1020102    Phnum    PñM  285900  1494700   

                         geometry  
0  POINT (287393.080 1495013.949)  
1  POINT (285493.095 1495013.949)

In [18]:
v_pt_gdf.isna().any()  # no NA at all

NUMBER      False
PHUMCODE    False
VILLAGE     False
PHUM        False
XPHUM       False
YPHUM       False
geometry    False
dtype: bool

In [22]:
v_pt_gdf[v_pt_gdf.PHUMCODE.duplicated()]  # 3 records have PHUMCODE = 0

NUMBER  PHUMCODE      VILLAGE      PHUM   XPHUM    YPHUM  \
13968       0         0  Kamlot Tmey   kMltfµI  359829  1267040   
13969       0         0    Pou Boeng  eBaF×bwg  369708  1271369   

                             geometry  
13968  POINT (359421.558 1267355.579)  
13969  POINT (369300.494 1271684.551)

In [44]:
v_pt_gdf[v_pt_gdf.PHUMCODE == 0]

NUMBER  PHUMCODE       VILLAGE       PHUM   XPHUM    YPHUM  \
13967       0         0  Traeung Soek  eRtIgEs¥k  365835        0   
13968       0         0   Kamlot Tmey    kMltfµI  359829  1267040   
13969       0         0     Pou Boeng   eBaF×bwg  369708  1271369   

                             geometry  
13967  POINT (365427.519 1269015.568)  
13968  POINT (359421.558 1267355.579)  
13969  POINT (369300.494 1271684.551)

#### explore matches of village codes: VillGis in CDB v.s. PHUMCODE in village points data

In [23]:
# set of village codes in points and CDB datasets
v_pt_set = set(v_pt_gdf.PHUMCODE)
v_CDB_set = set(CDB_v_df0.VillGis)

In [24]:
len(v_pt_set - v_CDB_set)  
# number of village codes in shapefile but not in spreadsheet

1762

In [25]:
len(v_CDB_set - v_pt_set)
# number of village codes in spreadsheet but not in shapefile

2232

#### left join based on village codes: CDB joined to village points

In [26]:
v_pt_w_CDB_gdf = v_pt_gdf.merge(
    CDB_v_df0[['VillGis', 'Village']],
    how='left',
    left_on='PHUMCODE',
    right_on='VillGis',
    indicator=True,
)

In [27]:
v_pt_w_CDB_gdf.head(2)

NUMBER PHUMCODE  VILLAGE   PHUM   XPHUM    YPHUM  \
0       1  1020101  Ou Thom  GUrFM  287800  1494700   
1       2  1020102    Phnum    PñM  285900  1494700   

                         geometry  VillGis  Village _merge  
0  POINT (287393.080 1495013.949)  1020101  Ou Thum   both  
1  POINT (285493.095 1495013.949)  1020102    Phnum   both

In [28]:
v_pt_w_CDB_gdf._merge.value_counts()  # village codes mostly matched upon first try

both          12206
left_only      1764
right_only        0
Name: _merge, dtype: int64

In [29]:
sum(v_pt_w_CDB_gdf.VILLAGE != v_pt_w_CDB_gdf.Village)  # but many village names not matched

5306

##### 12206-3542 code matched (village point with CDB joined), and village name matched

##### 3542 code matched (village point with CDB joined), but village name not matched

In [30]:
v_pt_w_CDB_gdf.loc[
    (v_pt_w_CDB_gdf.VILLAGE != v_pt_w_CDB_gdf.Village) & (v_pt_w_CDB_gdf._merge == 'both'),
    ['VILLAGE', 'Village']
]  
# 3542 spelling inconsistencies + mismatches of village names among the
# 12206 records with matching village codes

VILLAGE             Village
0                Ou Thom             Ou Thum
7                 Trabek             Trabaek
8                  Khile              Khilek
11         Dang Run Keut      Dang Run Kaeut
14     Prey Changha Keut  Prey Changha Kaeut
...                  ...                 ...
13958         Ou Andoung           O Andoang
13959      Ou Russei Leu      Ou Ressey Loeu
13960       Ou Kro Nhong       Ou Teuk Pleav
13961           Chikreng        Ou Chet Pram
13962            Ou Thom            Sras Pir

[3542 rows x 2 columns]

##### 1764 code not matched (village point without CDB joined)

In [40]:
v_pt_wo_CDB_gdf = v_pt_w_CDB_gdf[v_pt_w_CDB_gdf._merge == 'left_only']
# 1764 village points failed to be found in spreadsheet

In [42]:
set(v_pt_wo_CDB_gdf.PHUMCODE) == (v_pt_set - v_CDB_set)
# their 1764 codes form the same set as 
# the 1762 village codes in shapefile but not in spreadsheet

True

In [50]:
v_pt_set - v_CDB_set
# 1762 village codes in shapefile but not in spreadsheet

{0,
 20070403,
 20070404,
 20070405,
 20070406,
 11010104,
 11010105,
 9060408,
 12050501,
 12050502,
 12050503,
 12050504,
 12050505,
 12050506,
 12050507,
 12050508,
 12050509,
 12050510,
 12050511,
 12050512,
 12050513,
 12050514,
 12050515,
 12050516,
 12050517,
 12050518,
 12050519,
 12050520,
 12050521,
 12050522,
 12050523,
 8011901,
 8011902,
 8011903,
 5070314,
 1040514,
 5070316,
 12050601,
 12050602,
 12050603,
 12050604,
 12050605,
 12050606,
 12050607,
 12050608,
 12050609,
 20070607,
 20070608,
 20070609,
 11010307,
 11010308,
 5071119,
 11010405,
 12050901,
 12050902,
 12050903,
 12050904,
 12050905,
 12050906,
 12050907,
 12050908,
 12050909,
 12050910,
 12050911,
 2130410,
 2130411,
 2130412,
 2130413,
 2130414,
 8020501,
 8020502,
 8020503,
 2130509,
 8012401,
 8012402,
 8012403,
 8012404,
 8012405,
 8012406,
 8012407,
 8012408,
 12051101,
 12051102,
 12051103,
 12051104,
 12051105,
 12051106,
 12051107,
 12051108,
 2040612,
 2040614,
 2040618,
 2040619,
 2040620,
 20

In [43]:
v_pt_wo_CDB_gdf.PHUMCODE.value_counts()  # 3 village points have PHUMCODE = 0 
# 1764 = 1761 village codes + 3 zeros
# 1762 = the same 1761 codes+ 1 zero

0           3
8091201     1
10020103    1
10020102    1
10020101    1
           ..
3120216     1
3120215     1
3120214     1
3120213     1
3161805     1
Name: PHUMCODE, Length: 1762, dtype: int64

In [52]:
{len(str(code)) for code in v_pt_set - v_CDB_set}

{1, 7, 8}

### try to join commune boundaries with commune-level CDB data (cont.)

In [33]:
CDB_c_df0 = CDB_c_df_lst[0]

#### explore matches of commune codes

In [67]:
# set of commune codes in boundaries and CDB datasets
c_bnd_set = set(c_bnd_gdf.COMM_CODE)
c_CDB_set = set(CDB_c_df0.CommGis)

In [68]:
len(c_bnd_set - c_CDB_set)
# number of commune codes in shapefile but not in spreadsheet

91

In [69]:
len(c_CDB_set - c_bnd_set)
# number of commune codes in spreadsheet but not in shapefile

113

#### join

In [63]:
c_bnd_w_CDB_gdf = c_bnd_gdf.merge(
    CDB_c_df0[['CommGis', 'Commune']],
    how='left',
    left_on='COMM_CODE',
    right_on='CommGis',
    indicator=True,
)

In [64]:
c_bnd_w_CDB_gdf._merge.value_counts()  # commune codes mostly matched upon first try

both          1534
left_only       91
right_only       0
Name: _merge, dtype: int64

In [65]:
sum(c_bnd_w_CDB_gdf.COMM_NAME != c_bnd_w_CDB_gdf.Commune)  # but some commune names not matched

278

In [66]:
c_bnd_w_CDB_gdf.loc[
    (c_bnd_w_CDB_gdf.COMM_NAME != c_bnd_w_CDB_gdf.Commune) & (c_bnd_w_CDB_gdf._merge == 'both'),
    ['COMM_NAME', 'Commune']
]  
# 187 spelling inconsistencies + mismatches of commune names among the
# 1534 records with matching commune codes

COMM_NAME             Commune
2                Thma Kaev            Tma Kaev
5                    Kriel          Koun Kriel
7             Choam Khsant         Choam Ksant
19                 Morokut         Tuek Kraham
25              Bak Anlung         Bak Anloung
...                    ...                 ...
1601     Trapeang Kranhung  Trapeang Kranhoung
1609             Chak Krey                Bour
1611           Phnom Proek         Phnum Proek
1620  Preah Vihear  Temple             Morokot
1621                Sra Em            Sror Aem

[187 rows x 2 columns]